# Librerias utilizadas

In [ ]:
import pickle
import os, inspect
import pandas as pd
import numpy as np
import re, string
import json
from zipfile import ZipFile 
from json import dumps, loads, JSONEncoder, JSONDecoder
from os.path import basename, dirname

# Ruta del archivo

Recupera la ruta base donde nuestro script esta ubicado

In [ ]:
path_file = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))

# ObjectEncoder
Class PythonObjectEncoder permite traducir objetos de python (listas, diccionarios, enteros, ect.) en su equivalente json <br>
$ \small URL: $
https://www.w3schools.com/python/python_json.asp

In [ ]:
class PythonObjectEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (list, dict, str, int, float, bool, type(None))):
            return JSONEncoder.default(self, obj)
        return {'_python_object': pickle.dumps(obj)}

    def as_python_object(dct):
        if '_python_object' in dct:
            return pickle.loads(str(dct['_python_object']))
        return dct

# Entities
Clase encargada de transforma un excel.xlsx a .json. Una entidad es un conjunto que contiene objetos con atributos y propiedades similares. Ejemplo <br>
### Entidad autos, pizzas, seguros, departamentos
|$numeracion$| $\large Entidad$| $\large objetos$|
|---------|---------|---------|
|0|   $autos$ | kia, toyota, tesla, lamborgini|
|1|   $pizzas$ | pepperoni, salami, america, frutas|
|2|   $seguros$ | afocat, la positiva|
|3|   $departamentos$ | cusco, lima, arequipa, trujillo, ucayali, apurimac| <br>
$\large Observacion:$ cada objeto puede incluir un sinonimo

In [ ]:
class jsonEntities(object):
    @staticmethod
    def metadata(name, language="es", isOverridable=True, isEnum=False, isRegexp=False, automatedExpansion=False, allowFuzzyExtraction = False):
        return {
            #metadata
        "language" :language,
        "name" : name,
        "isOverridable" : isOverridable,
        "isEnum" : isEnum,# despliega los sinonimos en dialogflow
        "isRegexp" : isRegexp,
        "automatedExpansion" : automatedExpansion,
        "allowFuzzyExtraction" : allowFuzzyExtraction
        }
    @staticmethod
    def values(value, synonyms):
        return {
            "value":value,
            "synonyms":synonyms
        }

In [ ]:
class cEntities(object):
    def __init__(self, file):
        self._file=file
        self.eJson = jsonEntities()
        '''
            _values: [[_synonyms]]
            _synonyms:[[kia, sinonimo1, sinonimo2],[toyota, sinonimo1, sinonimo2]]
            _data:[autos:_values1, pizzas:_values2, seguros:_values3]
        '''
        self._values = []
        self._synonyms = []
        self._data = {}
    
    # #####################################################SETTERS#######################################################
    def _addSynonyms(self, synonym):
        self._synonyms.append(synonym)
        
    def _addValues(self, values):
        self._values.append(values)
    
    def _addData(self, key, values):
        if key not in self._data:
            self._data[key]=values
        else: 
            raise Exception('Ya existe una entidad con el mismo nombre {}'.format(key))
    
    ###################################################CLEAR DATA#######################################################
    def _clearSynonyms(self):
        self._synonyms = []
        
    def _clearValues(self):
        self._values = []
        
    
    ######################################################METODOS########################################################
    @staticmethod
    def makeDir(file, name='entities'):
        dirName = os.path.join(os.path.join(path_file,file),name)
        if not os.path.exists(dirName):
            os.mkdir(dirName)
        return dirName
            
    # Crea el directorio entities y adjunta los archivos 
    # entidad.json y entidad_entries_(idioma).json
    def _converToJson(self):
        _dir = self.makeDir(file=self._file)
        for name,data in self._data.items():
            # Dump entidad.json
            _dirEntidad = os.path.join(_dir,str(name)+'.json')
            with open(_dirEntidad, 'w', encoding='utf-8') as outfile:
                json.dump(self.eJson.metadata(name), outfile, cls=PythonObjectEncoder, indent=4, ensure_ascii=False)
            #Dump entidad_entries_(idioma).json
            p =[]
            for synm in data:
                p.append(self.eJson.values(synm[0],synm)) #establece en dialogflow el nombre de entities como sinonimos
                      
            _dirEntidad = os.path.join(_dir,str(name)+'_entries_es.json')
            with open(_dirEntidad, 'w', encoding='utf-8') as outfile:
                json.dump(p, outfile, cls=PythonObjectEncoder, indent=4, ensure_ascii=False)
                      
    # Extrae los datos de la pestaña entities del excel
    def _getData(self, excel):
        x = excel
        entities = x.iloc[:,0].dropna()
        x = x.fillna(0)
        for ent in range(0,len(entities.index)):
            if ent < len(entities.index)-1:
                synonyms = x.iloc[entities.index[ent] +1 :entities.index[ent+1], 1:]
            else:
                synonyms = x.iloc[entities.index[ent] +1 :, 1:]
                
            for i in range(len(synonyms.index)):
                for j in range(len(synonyms.columns)):
                    if synonyms.iat[i,j]!=0:
                        self._addSynonyms(str(synonyms.iat[i,j]))
                self._addValues(self._synonyms)
                self._clearSynonyms()
            self._addData(entities.iat[ent], self._values)
            self._clearValues()
    
    #Lee un excel
    @staticmethod
    def readExcel(url, sheet):
        '''
            url: path of the data.xls file
            sheet: sheet of the data.xls file
        '''
        return pd.read_excel(open(url, 'rb'), sheet_name=sheet)
                      
    # Call all methods
    def changeExcelToJson(self, url, sheet):
        excel = self.readExcel(url=url, sheet=sheet)
        self._getData(excel=excel)
        self._converToJson()

# Intents
Clase encargada de transforma un excel.xlsx a .json. Un intents es el flujo de conversacion que realiza entre un emisor y receptor, por ejemplo. <br>

$user: $ Hola <br>
$bot: $ Holaaaa!! <br>
$user: $ como estas? <br>
$bot: $ Muy bien, gracias <br>

In [ ]:
class jsonIntentsBot(object):
    @staticmethod
    def configuraciones(name, contexts, responses, auto=True, priority=500000, webhookUsed=False, webhookForSlotFilling=False, fallbackIntent=False, events=[]):
        return {
          "name": name,
          "auto": auto,
          "contexts": contexts,
          "responses": responses,
          "priority": priority,
          "webhookUsed": webhookUsed,
          "webhookForSlotFilling": webhookForSlotFilling,
          "fallbackIntent": fallbackIntent,
          "events": events}
    @staticmethod
    def responsePlatforms(defaultResponsePlatforms=1):
        if defaultResponsePlatforms==1:
            return {
                "facebook": True
            }
    @staticmethod
    def respuesta( affectedContexts, parameters, messages, resetContexts =False, defaultResponsePlatforms=1, speech =[]):      
        return [
            {
              "resetContexts": resetContexts,
              "affectedContexts": affectedContexts,
              "parameters": parameters,
              "messages": messages,
              "defaultResponsePlatforms": jsonIntentsBot.responsePlatforms(defaultResponsePlatforms),
              "speech": speech
            }
          ]
    @staticmethod
    def affectedContexts(name, parameters={}, lifespan=5):
        return {
          "name": name,
          "parameters": parameters,
          #lifespan funciona como una memoria, indica cuantos mensajes despues seguira almacenado la informacion
          "lifespan": lifespan
        }
    @staticmethod
    def parameter(dataType, name, value, required=False, isList=False):
        return {
              "required": required,
              "dataType": dataType,
              "name": name,
              "value": value,
              "isList": isList
        }
    @staticmethod
    def message(speech, types=0, lang="es"):
        return {
                  "type": types,
                  "lang": lang,
                  "speech": speech
                }
    @staticmethod
    def messageStyle0(speech, types=0, lang="es", platform="facebook"):
        return {
                  "type": types,
                  "platform": platform, # el mensaje instantaneo del bot es unicamente para la pltaforma facebook.
                  "lang": lang,
                  "speech": speech
                }
    @staticmethod
    def btnStyle1(text,postback):
        return {
            "text":text,
            "postback":postback
        }
    @staticmethod
    def messageStyle1(title, subtitle, img, btn, types, lang="es", platform="facebook"):
        return {
                  "type": types,
                  "platform": platform,
                  "lang": lang,
                  "title": title,
                  "subtitle": subtitle,
                  "imageUrl": img,
                  "buttons": btn
                }
    @staticmethod
    def messageStyle2(title, replies, types, lang="es", platform="facebook"):
        return {
              "type": types,
              "platform": platform,
              "lang": lang,
              "title": title,
              "replies": replies
        }
    @staticmethod
    def messageStyle3(img, types, lang="es", platform="facebook"):
        return {
          "type": types,
          "platform": platform,
          "lang": lang,
          "imageUrl": img
        }

In [ ]:
class cIntentsBot(object):
    def __init__(self, file):
        self._file = file
        self.iJson = jsonIntentsBot()
        
        #bot
        self._contexts = None
        self._parameters = []
        self._dataMessages = []
        self._affectedContexts =[]
        self._botJson ={}
        
    #############################################################METODOS BOT###############################################
    #SETTER
    def _addBotJson(self,key, botResponse):
        self._botJson[key] = botResponse
        
    def _addParameters(self, parameters):
        self._parameters.extend(parameters)
        
    def _addAffectedContexts(self, affectedContexts):
        self._affectedContexts.append(affectedContexts)
        
    def _addDataMessages(self, speech, styles):
        self._dataMessages.append(self.iJson.message(speech))
        for style in styles:
            self._dataMessages.append(style)
            
    def _addffectedContexts(self, contextOut):
        if contextOut!=0:
            for name in str(contextOut).split(','):
                self._addAffectedContexts(self.iJson.affectedContexts(name.strip()))
    
    #estilos de respuesta para fb
    def _processStyle(self, message, types):
        split = [message.strip() for message in message.split('|')]
        if types == 0:
            return self.iJson.messageStyle0(speech=split, types=int(types))
        if types == 1:
            #split = message.split('|')
            title = split[0]
            subtitle=split[1]
            imgUrl =split[2]
            btn = []
            for i in range(0,len(split[3:]),2):
                if split[i]!= None:
                    btn.append(self.iJson.btnStyle1(text=split[i+3],postback=split[i+4]))
            return self.iJson.messageStyle1(title=title, subtitle=subtitle, img=imgUrl, btn=btn, types=int(types))
        if types == 2:
            #split = message.split('|')
            title = split[0]
            replies = []
            for i in range(0,len(split[1:])):
                replies.append(split[i+1])
            return self.iJson.messageStyle2(title=title, replies=replies, types=int(types))
        if types ==3:
            return self.iJson.messageStyle3(img=split,types=int(types))
        
    #CLEAR VALUES
    def _clear(self):
        self._contexts = None
        self._parameters = []
        self._dataMessages = []
        self._affectedContexts =[]
        
    #METODOS   
    
    def _addContexts(self, contextIn):
        if contextIn!=0:
            contextIn =[context.strip() for context in str(contextIn).split(',')]
            self._contexts = contextIn
        else:
            self._contexts = []
            
    ''' convierte los atributos de bot a  intents/name(intents).json '''  
    def _converToJsonBot(self):
        _dir = self.makeDir(file=self._file)
        for key, intentBot in self._botJson.items():
            _dirBot=os.path.join(_dir,'%s.json'%str(key))
            with open(_dirBot, 'w', encoding='utf-8') as outfile:
                json.dump(intentBot, outfile, cls=PythonObjectEncoder, indent=4, ensure_ascii=False)
                
    ''' busca en la columna mensaje del bot del archivo.xlsx si existe una frase con etiqueta especial $ 
    y devuelve esos valores''' 
    @staticmethod
    def _getEntities(userMessage, entitie='$'):
        message = []
        userMessage = np.array(userMessage.split())
        index = [i for i, split in enumerate(userMessage) if split.startswith(entitie,0,1)]
        return list(userMessage[index])
    
    ''' Obtiene los datos del excel unicamente centrandose en los campos bot,
    mensaje del bot, mensaje instantaneo, style y contextos''' 
    def _getDataToBot(self, intents, x, ent):
        self._addffectedContexts(x.iat[intents.index[ent],7])
        self._addContexts(x.iat[intents.index[ent],6])
        if ent < len(intents.index)-1:
            bot_message = x.iloc[intents.index[ent]+1 :intents.index[ent+1], 3:8]
        else:
            bot_message = x.iloc[intents.index[ent]+1:, 3:8]
            
        speech = []
        style = []
        dup = {}
        for i in range(len(bot_message.index)):
            #user message
            bot = str(bot_message.iat[i,0])
            if bot != "0":
                speech.append(bot.replace('sys.', ''))
                parameters = self._getEntities(bot)
                if len(parameters)>0:
                    params = []
                    for parameter in parameters:
                        p = parameter.strip()
                        aux_p = re.sub('['+string.punctuation+']', '', p[-2:])
                        p = p[0:-2]+aux_p
                        if p[1:] in dup:
                            pass
                        else:
                            if p.startswith('sys',1):
                                params.append(self.iJson.parameter(dataType='@'+p[1:], name=p[5:], value='$'+p[5:]))
                            else:
                                params.append(self.iJson.parameter(dataType='@'+p[1:], name=p[1:], value='$'+p[1:]))
                            dup[p[1:]] = 0
                        
                    self._addParameters(params)
            #style facebook response
            f_bot = str(bot_message.iat[i,1])
            if f_bot != "0":
                if str(bot_message.iat[i,2]) is not None:
                    style.append(self._processStyle(f_bot,bot_message.iat[i,2]))
        self._addDataMessages(speech,style)
        self._addBotJson(str(intents.iat[ent]), self.iJson.configuraciones(name=str(intents.iat[ent]),
                                                 contexts = self._contexts, 
                                                 responses=self.iJson.respuesta(affectedContexts=self._affectedContexts,
                                                                           parameters=self._parameters,
                                                                           messages= self._dataMessages,
                                                                           defaultResponsePlatforms=1)))
        self._clear()   
    

    
    ''' recorre de intents en intents de la pestaña intents del archivo excel'''   
    def _getData(self, excel):
        x = excel
        intents = x.iloc[:,0].dropna()
        x = x.fillna(0)
        for ent in range(0,len(intents.index)):
            self._getDataToBot(intents, x, ent)  
            
    ''' llama a todo los  modulos involucrados'''           
    def changeExcelToJson(self, url, sheet):
        excel = self.readExcel(url=url, sheet=sheet)
        self._getData(excel=excel)
        self._converToJsonBot()
        
    @staticmethod
    def readExcel(url, sheet):
        '''
            url: path of the data.xls file
            sheet: sheet of the data.xls file
        '''
        return pd.read_excel(open(url, 'rb'), sheet_name=sheet)
    
    @staticmethod
    def makeDir(file, name='intents'):
        dirName = os.path.join(os.path.join(path_file, file), name)
        if not os.path.exists(dirName):
            os.mkdir(dirName)
        return dirName

In [ ]:
class jsonIntentsUser(object):
    @staticmethod
    def messages(messages,isTemplate=False, count=0):
        return {
            "data": messages,
            "isTemplate": isTemplate,
            "count": count
          }
    @staticmethod
    def messageData(text, alias=None, meta=None, userDefined=False):
        if userDefined:
            return {
                "text": text,
                "alias": alias,
                "meta": meta,
                "userDefined": userDefined
              }
        else:
            return {
                "text": text,
                "userDefined": userDefined
              }

In [ ]:
class cIntentsUser(object):
    #################################################USER########################################################## 
    def __init__(self, file):
        self._file = file
        self.iJson = jsonIntentsUser()
        #user
        self._userResponse = []
        self._messages = []
        self._userJson = {}
        
    #SETTER
    def _addUserJson(self, key, userResponse):
        self._userJson[key] = userResponse
        
    def _addUserResponse(self, UserMessages):
        self._userResponse.append(UserMessages)
    
    def _clearUserResponse(self):
        self._userResponse = []
        
    def _addMessages(self, menssage):
        self._messages.append(menssage)

    # CLEAR
    def _clearMessages(self):
        self._messages = []
    
    #METODOS
    ''' parsea la columa mensaje usuario del archivo.xlsx de acuerdo a la columna entidades
      ejemplo:
      mensaje usuario: hola buenos dias
      entidades : transicion-dias
      
      retorna : [hola buenos][dias]
      '''
    @staticmethod    
    def _splitString(userMessage, entitie):
        message = []
        userMessage = userMessage.split()
        index = [i for i, split in enumerate(userMessage) if entitie.lower() in split.lower()]
        aux = 0
        for x in index:
            if x!=aux:
                message.append(str(' '.join(userMessage[aux:x])).strip())
                message.append(' ')
                message.append(userMessage[x])
                message.append(' ')
            else:
                message.append(userMessage[x])
                message.append(' ')
            aux = x+1
        if aux < len(userMessage):
            message.append(str(' '.join(userMessage[aux:])).strip())
            message.append(' ')
        return message
    
    @staticmethod
    def _getAllEntities(user, entities):
        _words = [user]
        _aux = []
        _entities = entities.split(',')
        if len(_entities)>0:
            for _ent in _entities:
                split = _ent.split('_')
                for word in _words:
                    _message = cIntentsUser._splitString(word, str(split[1].strip()))
                    _aux.extend(_message)
                _words = _aux
                _aux = []
        return _words
                
                    
                
        
    ''' convierte los atributos de bot a  intents/name(intents)_usersays_es.json'''
    def _converToJsonUser(self):
        _dir = self.makeDir(file=self._file)
        for key, intentUser in self._userJson.items():
            _dirUser=os.path.join(_dir,'%s_usersays_es.json'%str(key))
            with open(_dirUser, 'w', encoding='utf-8') as outfile:
                json.dump(intentUser, outfile, cls=PythonObjectEncoder, indent=4,ensure_ascii=False)
                
    ''' Obtiene los datos del excel unicamente centrandose en los campos user,
    mensaje usuario y entidades'''             
    def _getDataToUser(self, intents, x, ent):
        if ent < len(intents.index)-1:
            user_message = x.iloc[intents.index[ent] +1:intents.index[ent+1], 1:3]
        else:
            user_message = x.iloc[intents.index[ent] +1:, 1:3]
        for i in range(len(user_message.index)):
            user = str(user_message.iat[i,0])
            if user_message.iat[i,1]!=0:
                entities = str(user_message.iat[i,1])
                split = entities.split(',')
                for text in self._getAllEntities(user,entities):
                    bandera = False
                    for s in split:
                        s = s.split('_')
                        split0 = str(s[0]).strip()
                        split1 = str(s[1]).strip()
                        if split1.lower() in text.strip().lower():
                            alias = split0
                            if alias.startswith('sys'):
                                alias = alias[4:]
                            #print(alias)
                            self._addMessages(self.iJson.messageData(text, alias=alias, meta="@"+split0, userDefined=True))
                            bandera = True
                            break
                    if not bandera:
                        #print(text)
                        self._addMessages(self.iJson.messageData(text))
                self._addUserResponse(self.iJson.messages(messages=self._messages))
                self._clearMessages()
            else:
                self._addMessages(self.iJson.messageData(text=user))
                self._addUserResponse(self.iJson.messages(messages=self._messages))
                self._clearMessages()
        self._addUserJson(str(intents.iat[ent]), self._userResponse)
        self._clearUserResponse()      
    
    ''' recorre de intents en intents de la pestaña intents del archivo excel'''   
    def _getData(self, excel):
        x = excel
        intents = x.iloc[:,0].dropna()
        x = x.fillna(0)
        for ent in range(0,len(intents.index)):
            self._getDataToUser(intents,x,ent)
            
    ''' llama a todo los  modulos involucrados'''           
    def changeExcelToJson(self, url, sheet):
        excel = self.readExcel(url=url, sheet=sheet)
        self._getData(excel=excel)
        self._converToJsonUser()
        
    @staticmethod
    def readExcel(url, sheet):
        '''
            url: path of the data.xls file
            sheet: sheet of the data.xls file
        '''
        return pd.read_excel(open(url, 'rb'), sheet_name=sheet)
    
    @staticmethod
    def makeDir(file, name='intents'):
        dirName = os.path.join(os.path.join(path_file, file), name)
        if not os.path.exists(dirName):
            os.mkdir(dirName)
        return dirName

In [ ]:
class cPackage(object):
    def __init__(self, version, file):
        self.version = version
        self.file = file
    def _jsonFormatPackage(self):
        return {
            "version":str(self.version)
        }
    def _converToJsonPackage(self):
        dirPackage = os.path.join(os.path.join(path_file, self.file),'package.json')
        with open(dirPackage, 'w') as outfile:
            json.dump(self._jsonFormatPackage(), outfile)

# Zipdata and main

In [ ]:
def get_all_file_paths(directory): 
  
    # initializing empty file paths list 
    file_paths = [] 
  
    # crawling through directory and subdirectories 
    for root, directories, files in os.walk(directory): 
        for filename in files: 
            # join the two strings in order to form the full filepath. 
            filepath = os.path.join(root, filename) 
            file_paths.append(filepath) 
  
    # returning all file paths 
    return file_paths         

In [ ]:
def main(ruta , sheet):
    #nombre del directorio donde se almacenara los json y luego comprimir
    file = 'dialogflow_model'
    #creamos el directorio bot
    dirName = os.path.join(path_file,file)
    if not os.path.exists(dirName):
        os.mkdir(dirName)
    
    #inicializamos las clases
    package = cPackage('1.0.0', file)
    entities = cEntities(file)
    intentsUser = cIntentsUser(file)
    intentsBot = cIntentsBot(file)
    
    #ejecutamos las instrucciones
    package._converToJsonPackage()
    entities.changeExcelToJson(ruta,sheet[0])
    intentsUser.changeExcelToJson(ruta,sheet[1])
    intentsBot.changeExcelToJson(ruta,sheet[1])
    
    # Obtener los nombres de los archivos a comprimir
    file_paths = get_all_file_paths(dirName) 
  
    # Imprimir la lista de archivos a comprimir
    print('Archivos a comprimir:') 
    for file_name in file_paths: 
        print(file_name) 
    
    # writing files to a zipfile 
    with ZipFile(dirName+'/dialogflow.zip','w') as zip: 
        # writing cada archivo
        for file in file_paths: 
            if basename(file) == 'package.json':
                zip.write(file, basename(file))
            else:
                zip.write(file, basename(dirname(file))+'/'+basename(file))
  
    print('successfully complete!')

In [ ]:
entities=pd.read_csv('Entities-Grid view.csv')
intents=pd.read_csv('Intents-Grid view.csv')
with pd.ExcelWriter('output.xlsx') as writer:  # doctest: +SKIP
    entities.to_excel(writer, sheet_name='Entities', index=False)
    intents.to_excel(writer, sheet_name='Intents', index=False)

In [ ]:
if __name__ == "__main__":
    #ruta del excel
    ruta = input("Ingrese la ruta de su archivo output.xlsx")
    if ruta !="":
        ruta = ruta+"/output.xlsx"
        #pestañas del excel, primero las entidades y segundo los intents
        sheet = ['Entities', 'Intents']
        main(ruta, sheet)
    else:
        print("Ingrese una ruta valida")